# Convolutional Neural Networks for Image Recognition
## Advanced Machine Learning

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import fnmatch
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Convolution2D, Dense, MaxPooling2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras import Sequential
import random
import re

# pip install pypng
import png
import sklearn.metrics

# Provided 
from core50 import *

##################
# Configure figure parameters

FONTSIZE = 18
FIGURE_SIZE = (10,4)
FIGURE_SIZE2 = (10,10)

plt.rcParams.update({'font.size': FONTSIZE})
plt.rcParams['figure.figsize'] = FIGURE_SIZE
# Default tick label size
plt.rcParams['xtick.labelsize'] = FONTSIZE
plt.rcParams['ytick.labelsize'] = FONTSIZE

In [ ]:
# Load up a single image & view it
directory = '/home/fagg/datasets/core50/core50_128x128/s1/o21'
files = os.listdir(directory)

r = png.Reader(directory + "/" + files[0])
it = r.read()
image_2d = np.vstack(map(np.uint8, it[2]))
image_3d = np.reshape(image_2d,
                         (128,128,3))

plt.imshow(image_3d)

## Load Data Sets

In [ ]:
## File location
directory_base = '/home/fagg/datasets/core50/core50_128x128'

# Training set: define which files to load for each object
#val = '.*[05].png'
val_files = '.*[0].png'

### Positive cases
# Define which objects to load
#object_list = ['o25', 'o22', 'o23', 'o24']
object_list = ['o21']

# Define which conditions to load
#condition_list = ['s1', 's2', 's3', 's4', 's5', 's7', 's8', 's9', 's10', 's11']
#condition_list = ['s1', 's2', 's3', 's4']
condition_list = ['s1']

# Load all of the objects/condition
ins_pos = load_multiple_image_sets_from_directories(directory_base, condition_list, object_list, val_files)

### Negative cases
# Define which objects to load
#object_list2 = ['o45', 'o42', 'o43', 'o44']
object_list2 = ['o41']
ins_neg = load_multiple_image_sets_from_directories(directory_base, condition_list, object_list2, val_files)

### Combine positives and negatives into a common data set
outs_pos = np.append(np.ones((ins_pos.shape[0],1)), np.zeros((ins_pos.shape[0],1)), axis=1)
outs_neg = np.append(np.zeros((ins_pos.shape[0],1)), np.ones((ins_pos.shape[0],1)), axis=1)

ins = np.append(ins_pos, ins_neg, axis=0)
outs = np.append(outs_pos, outs_neg, axis=0)

########################################################################
# Validation set
# Define which files to load for each object
val_files = '.*[5].png'

### Positives
# Define which objects to load
object_list = ['o21']
#object_list = ['o21']

# Load the positives
ins_pos_validation = load_multiple_image_sets_from_directories(directory_base, condition_list, object_list, val_files)

### Negatives
# Define objects
object_list2 = ['o41']
#object_list2 = ['o41']

# Load the negative cases
ins_neg_validation = load_multiple_image_sets_from_directories(directory_base, condition_list, object_list2, val_files)

### Combine positives and negatives
outs_pos_validation = np.append(np.ones((ins_pos_validation.shape[0], 1)), np.zeros((ins_pos_validation.shape[0], 1)), axis=1)
outs_neg_validation = np.append(np.zeros((ins_pos_validation.shape[0], 1)), np.ones((ins_pos_validation.shape[0], 1)), axis=1)

ins_validation = np.append(ins_pos_validation, ins_neg_validation, axis=0)
outs_validation = np.append(outs_pos_validation, outs_neg_validation, axis=0)

In [ ]:
ins.shape

In [ ]:
outs.shape

In [ ]:
ins_validation.shape

In [ ]:
outs_validation.shape

## Create Network

In [ ]:
def create_classifier_network(image_size, nchannels, n_classes, lambda_l2=.0001, p_dropout=0.5):
    model = Sequential()
    model.add(InputLayer(input_shape=(image_size[0], image_size[1], nchannels)))
   
    ### Fill in detail
    
    print(model.summary())
    
    return model


In [ ]:
model = create_classifier_network((ins.shape[1], ins.shape[2]), ins.shape[3], 2)

In [ ]:
# Callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True,
                                                      min_delta=0.01)

In [ ]:
history = model.fit(x=ins, y=outs, epochs=100, verbose=1,
                        validation_data=(ins_validation, outs_validation), 
                        callbacks=[early_stopping_cb])

## Evaluation

In [ ]:
def generate_roc(model, ins, outs, ins_validation, outs_validation):
    '''
    Produce a ROC plot given a model, a set of inputs and the true outputs
    
    Assume that model produces N-class output; we will only look at the class 0 scores
    '''
    # Compute probabilistic predictions given images
    pred = model.predict(ins)
    # Compute false positive rate & true positive rate + AUC
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(outs[:,0], pred[:,0])
    auc = sklearn.metrics.auc(fpr, tpr)
    
    # Compute probabilistic predictions given images
    pred_val = model.predict(ins_validation)
    # Compute false positive rate & true positive rate + AUC
    fpr_val, tpr_val, thresholds_val = sklearn.metrics.roc_curve(outs_validation[:,0], pred_val[:,0])
    auc_val = sklearn.metrics.auc(fpr_val, tpr_val)
    
    
    # Generate the plot
    plt.figure(1)
    plt.axis('equal')
    plt.plot([0,1], [0,1], 'k--')
    plt.plot(fpr, tpr, 'b', label='Train AUC = {:.3f}'.format(auc))
    plt.plot(fpr_val, tpr_val, 'r', label='Validation AUC = {:.3f}'.format(auc_val))
    plt.legend(loc='best')
    plt.xlabel('FPR', fontsize=FONTSIZE)
    plt.ylabel('TPR', fontsize=FONTSIZE)
    

In [ ]:
generate_roc(model, ins, outs, ins_validation, outs_validation)

## Visualize Model Internals

In [ ]:
def intermediate_model_state(model, ins, layer_list):
    '''
    Return layer activations for intermediate layers in a model for a set of examples
    
    :param model: Model in question
    :param ins: Input tensor (examples, rows, cols, channels)
    :param layer_list: List of layer names to produce activations for
    :returns: a list of numpy arrays
    '''
    # Translate layer names into corresponding output tensors
    layer_outputs = [l.output for l in model.layers if l.name in layer_list]
    
    # Construct a new Keras model that outputs these tensors
    # The internal structure of the model itself is referenced through the input and output tensor lists
    new_model = keras.models.Model(inputs=model.input, outputs=layer_outputs)
    
    # Evaluate the new model
    activations = new_model.predict(ins_validation)
    
    # Return a list of activation numpy arrays
    return activations

def visualize_state(activations, width=1, example=0, cmap='plasma'):
    '''
    Produce graphical representation of a set of image channels
    
    :param activations: numpy array (example, rows, cols, channels)
    :param width: Number of images displayed horizontally
    :param example: Index of example to display
    :param cmap: Color map to use for plotting
    '''
    # Size of the individual images
    nrows = activations.shape[1]
    ncols = activations.shape[2]
    # Number of channels
    nfilters = activations.shape[3]
    
    # Tile all of the sub-images 
    grid = np.zeros((int((nfilters-1)/width + 1) * nrows, ncols * width))
    
    # Loop over image
    for i in range(nfilters):
        # Compute r,c of tile to place the ith image into
        r = int(i / width)
        c = i % width
        grid[nrows*r: nrows*(r+1), ncols*c:ncols*(c+1)] = activations[example,:,:,i]
        
    # Plot
    plt.matshow(grid, cmap=cmap) 

In [ ]:
# Compute activations for 2 layers over a set of examples
layer_list=['C1']
activations = intermediate_model_state(model, ins_validation, layer_list)

In [ ]:
# Plot convolutional layers 1 and 2
example=40
plt.imshow(ins_validation[example,:,:,:])
visualize_state(activations, width=10, example=example)
#visualize_state(activations[0], width=10, example=example)
#visualize_state(activations[1], width=20, example=example)
#visualize_state(activations[2], width=20, example=example)
#visualize_state(activations[3], width=30, example=example)